In [11]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [12]:
import pandas as pd
import pylab as pl
import numpy as np
import geopandas as gp
import pickle

#NYC Census tracts vs. School districts

In [13]:
#Loading census tracts shape file 
tracts = gp.GeoDataFrame.from_file('gz_2010_36_140_00_500k.shp')
tracts_loc = gp.GeoSeries(tracts['geometry'])

#Loading school district shapefile
districts = gp.GeoDataFrame.from_file("nysd.shp")

#Unifying epsg's
tracts = tracts.to_crs(epsg=4326)
districts = districts.to_crs(epsg=4326)

In [14]:
tracts.head()

,CENSUSAREA,COUNTY,GEO_ID,LSAD,NAME,STATE,TRACT,geometry
0,0.914,001,1400000US36001000100,Tract,1,36,000100,"POLYGON ((-73.741265 42.678681, -73.736974 42...."
1,0.797,001,1400000US36001000200,Tract,2,36,000200,"POLYGON ((-73.75022299999999 42.653544, -73.75..."
2,3.482,001,1400000US36001000401,Tract,4.01,36,000401,"POLYGON ((-73.83850799999999 42.695152, -73.84..."
3,0.211,001,1400000US36001000501,Tract,5.01,36,000501,"POLYGON ((-73.77741999999999 42.664931, -73.77..."
4,0.225,001,1400000US36001001500,Tract,15,36,001500,"POLYGON ((-73.78650399999999 42.653026, -73.78..."


In [15]:
districts.head()

,SchoolDist,Shape_Area,Shape_Leng,geometry
0,13,1.048871e+08,86613.431251,POLYGON ((-73.97906084845808 40.70594602847149...
1,15,1.961494e+08,153416.270604,POLYGON ((-73.98633134976369 40.69105051016147...
2,12,6.903543e+07,48525.965820,"POLYGON ((-73.88284445508626 40.8478172264843,..."
3,30,3.182050e+08,151006.383911,POLYGON ((-73.90647314610099 40.79018117524141...
4,10,2.825398e+08,94971.139091,POLYGON ((-73.86789043713642 40.90298695418561...


In [16]:
#Finding centroids for tracts, so that they all fit into only one district
tracts['centroid'] = gp.GeoSeries(tracts['geometry'].centroid)
tracts.head()

,CENSUSAREA,COUNTY,GEO_ID,LSAD,NAME,STATE,TRACT,geometry,centroid
0,0.914,001,1400000US36001000100,Tract,1,36,000100,"POLYGON ((-73.741265 42.678681, -73.736974 42....",POINT (-73.73627075553357 42.66894510898201)
1,0.797,001,1400000US36001000200,Tract,2,36,000200,"POLYGON ((-73.75022299999999 42.653544, -73.75...",POINT (-73.75150755824929 42.66437580394736)
2,3.482,001,1400000US36001000401,Tract,4.01,36,000401,"POLYGON ((-73.83850799999999 42.695152, -73.84...",POINT (-73.86485481849689 42.70630965323624)
3,0.211,001,1400000US36001000501,Tract,5.01,36,000501,"POLYGON ((-73.77741999999999 42.664931, -73.77...",POINT (-73.78241589934387 42.6703347193544)
4,0.225,001,1400000US36001001500,Tract,15,36,001500,"POLYGON ((-73.78650399999999 42.653026, -73.78...",POINT (-73.78167580463717 42.65750207435287)


In [17]:
#Adding school district to each tract

tracts['district'] = np.zeros(len(tracts))

for t in range(len(tracts)):   

    for d in range(len(districts)):
        if tracts['centroid'][t].intersects(districts['geometry'][d]) == True:
            tracts['district'][t] = districts['SchoolDist'][d]
            
tracts.head()

/usr/local/lib/python2.7/site-packages/IPython/kernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,CENSUSAREA,COUNTY,GEO_ID,LSAD,NAME,STATE,TRACT,geometry,centroid,district
0,0.914,001,1400000US36001000100,Tract,1,36,000100,"POLYGON ((-73.741265 42.678681, -73.736974 42....",POINT (-73.73627075553357 42.66894510898201),0
1,0.797,001,1400000US36001000200,Tract,2,36,000200,"POLYGON ((-73.75022299999999 42.653544, -73.75...",POINT (-73.75150755824929 42.66437580394736),0
2,3.482,001,1400000US36001000401,Tract,4.01,36,000401,"POLYGON ((-73.83850799999999 42.695152, -73.84...",POINT (-73.86485481849689 42.70630965323624),0
3,0.211,001,1400000US36001000501,Tract,5.01,36,000501,"POLYGON ((-73.77741999999999 42.664931, -73.77...",POINT (-73.78241589934387 42.6703347193544),0
4,0.225,001,1400000US36001001500,Tract,15,36,001500,"POLYGON ((-73.78650399999999 42.653026, -73.78...",POINT (-73.78167580463717 42.65750207435287),0


In [18]:
tracts = tracts[tracts['district']!=0]
tracts = tracts.reset_index(drop=True)
len(tracts)

2114

In [19]:
#Creating dictionary for school districts

districts = {}

for i in range(len(tracts)):
    districts[str(tracts['GEO_ID'][i])] =  tracts['district'][i]

#Household income

In [20]:
#HC01_EST_VC02 ------Total; Estimate; Households
#HC02_EST_VC02 ------Median income (dollars); Estimate; Households
#HC01_EST_VC04 ------Total; Estimate; Households - One race-- - White
#HC02_EST_VC04 ------Median income (dollars); Estimate; Households - One race-- - White
#HC01_EST_VC05 ------Total; Estimate; Households - One race-- - Black or African American
#HC02_EST_VC05 ------Median income (dollars); Estimate; Households - One race-- - Black or African American
#HC01_EST_VC06 ------Total; Estimate; Households - One race-- - American Indian and Alaska Native
#HC02_EST_VC06 ------Median income (dollars); Estimate; Households - One race-- - American Indian and Alaska Native
#HC01_EST_VC07 ------Total; Estimate; Households - One race-- - Asian
#HC02_EST_VC07 ------Median income (dollars); Estimate; Households - One race-- - Asian
#HC01_EST_VC08 ------Total; Estimate; Households - One race-- - Native Hawaiian and Other Pacific Islander
#HC02_EST_VC08 ------Median income (dollars); Estimate; Households - One race-- - Native Hawaiian and Other Pacific Islander
#HC01_EST_VC09 ------Total; Estimate; Households - One race-- - Some other race
#HC02_EST_VC09 ------Median income (dollars); Estimate; Households - One race-- - Some other race
#HC01_EST_VC10 ------Total; Estimate; Households - Two or more races
#HC02_EST_VC10 ------Median income (dollars); Estimate; Households - Two or more races
#HC01_EST_VC12 ------Total; Estimate; Hispanic or Latino origin (of any race)
#HC02_EST_VC12 ------Median income (dollars); Estimate; Hispanic or Latino origin (of any race)
#HC01_EST_VC13 ------Total; Estimate; White alone, not Hispanic or Latino
#HC02_EST_VC13 ------Median income (dollars); Estimate; White alone, not Hispanic or Latino
#HC01_EST_VC30 ------Total; Estimate; NONFAMILY HOUSEHOLDS - Nonfamily households
#HC02_EST_VC30 ------Median income (dollars); Estimate; NONFAMILY HOUSEHOLDS - Nonfamily households
#HC01_EST_VC31 ------Total; Estimate; NONFAMILY HOUSEHOLDS - Nonfamily households - Female householder
#HC02_EST_VC31 ------Median income (dollars); Estimate; NONFAMILY HOUSEHOLDS - Nonfamily households - Female householder
#HC01_EST_VC33 ------Total; Estimate; NONFAMILY HOUSEHOLDS - Nonfamily households - Female householder - Not living alone
#HC02_EST_VC33 ------Median income (dollars); Estimate; NONFAMILY HOUSEHOLDS - Nonfamily households - Female householder - Not living alone
#HC01_EST_VC34 ------Total; Estimate; NONFAMILY HOUSEHOLDS - Nonfamily households - Male householder
#HC02_EST_VC34 ------Median income (dollars); Estimate; NONFAMILY HOUSEHOLDS - Nonfamily households - Male householder
#HC01_EST_VC36 ------Total; Estimate; NONFAMILY HOUSEHOLDS - Nonfamily households - Male householder - Not living alone
#HC02_EST_VC36 ------Median income (dollars); Estimate; NONFAMILY HOUSEHOLDS - Nonfamily households - Male householder - Not living alone

In [21]:
#Loading median income data

median_income=pd.read_csv('median_income.csv')
median_income.head()

,GEO.id,GEO.id2,GEO.display-label,HC01_EST_VC02,HC01_MOE_VC02,HC02_EST_VC02,HC02_MOE_VC02,HC01_EST_VC04,HC01_MOE_VC04,HC02_EST_VC04,...,HC02_EST_VC33,HC02_MOE_VC33,HC01_EST_VC34,HC01_MOE_VC34,HC02_EST_VC34,HC02_MOE_VC34,HC01_EST_VC36,HC01_MOE_VC36,HC02_EST_VC36,HC02_MOE_VC36
0,1400000US36005000100,36005000100,"Census Tract 1, Bronx County, New York",0,16,-,**,-,**,-,...,-,**,-,**,-,**,-,**,-,**
1,1400000US36005000200,36005000200,"Census Tract 2, Bronx County, New York",1379,77,74837,18514,32.1,7.1,92692,...,-,**,32.9,19.2,8796,19654,5.6,8.5,-,**
2,1400000US36005000400,36005000400,"Census Tract 4, Bronx County, New York",1819,80,77991,5195,28,6.8,76900,...,56518,20039,46.5,16.1,68295,48585,14.8,13.4,113405,59704
3,1400000US36005001600,36005001600,"Census Tract 16, Bronx County, New York",1912,105,32354,4876,33.6,7,35000,...,-,**,26.7,11.2,23320,5149,0,4.9,-,**
4,1400000US36005001900,36005001900,"Census Tract 19, Bronx County, New York",834,60,34635,4017,34.3,8.1,29896,...,57188,36171,55.7,12.4,27065,16647,10.1,6.1,110250,146647


In [22]:
#Adding Households and Median income (dollars); renaming columns:

district_income = median_income[['GEO.id', 'HC01_EST_VC02', 'HC02_EST_VC02']]
district_income = district_income.rename(columns = {'HC01_EST_VC02':'Households', 
                                                    'HC02_EST_VC02':'Median income (dollars)'})

#Adding district

district_income['District'] = district_income['GEO.id'].map(districts)
district_income = district_income[district_income['District']>0]
district_income = district_income[district_income['Households']>0]
district_income = district_income.reset_index(drop=True)

district_income.head()

,GEO.id,Households,Median income (dollars),District
0,1400000US36005000400,1819,77991,8
1,1400000US36005001600,1912,32354,8
2,1400000US36005001900,834,34635,7
3,1400000US36005002000,2980,23423,8
4,1400000US36005002300,1883,14304,7


In [23]:
#Converting income to float
def to_number(s):
    try:
        s1 = float(s)
        return s1
    except ValueError:
        return 0

district_income['Median income (dollars)'] = map(lambda x: to_number(x), district_income['Median income (dollars)'])
district_income = district_income[district_income['Median income (dollars)']>0]

print len(district_income)
district_income.head()

2063


,GEO.id,Households,Median income (dollars),District
0,1400000US36005000400,1819,77991,8
1,1400000US36005001600,1912,32354,8
2,1400000US36005001900,834,34635,7
3,1400000US36005002000,2980,23423,8
4,1400000US36005002300,1883,14304,7


In [43]:
#Finding the median income of each district

socioeconomic = pd.DataFrame()
socioeconomic['District'] = range(1, 33)
socioeconomic = socioeconomic.set_index('District')

socioeconomic['Median income (dollars)'] = np.zeros(32)
for i in range(1,33):
    socioeconomic['Median income (dollars)'][i] = district_income[district_income['District'] == i]['Median income (dollars)'].median()

In [44]:
socioeconomic

,Median income (dollars)
District,
1,62768.0
2,106625.0
3,88067.5
4,26578.0
5,35583.0
6,36999.5
7,21959.0
8,44840.0
9,24134.5


#Educational attainment

In [26]:
#HC01_EST_VC01------Total; Estimate; Population 18 to 24 years
#HC01_EST_VC02------Total; Estimate; Less than high school graduate
#HC01_EST_VC03------Total; Estimate; High school graduate (includes equivalency)
#HC01_EST_VC04------Total; Estimate; Some college or associate's degree
#HC01_EST_VC05------Total; Estimate; Bachelor's degree or higher
#HC01_EST_VC07------Total; Estimate; Population 25 years and over
#HC01_EST_VC08------Total; Estimate; Population 25 years and over - Less than 9th grade
#HC01_EST_VC09------Total; Estimate; Population 25 years and over - 9th to 12th grade, no diploma
#HC01_EST_VC10------Total; Estimate; Population 25 years and over - High school graduate (includes equivalency)
#HC01_EST_VC11------Total; Estimate; Population 25 years and over - Some college, no degree
#HC01_EST_VC12------Total; Estimate; Population 25 years and over - Associate's degree
#HC01_EST_VC13------Total; Estimate; Population 25 years and over - Bachelor's degree
#HC01_EST_VC14------Total; Estimate; Population 25 years and over - Graduate or professional degree
#HC01_EST_VC16------Total; Estimate; Percent high school graduate or higher
#HC01_EST_VC17------Total; Estimate; Percent bachelor's degree or higher
#HC01_EST_VC19------Total; Estimate; Population 25 to 34 years
#HC01_EST_VC20------Total; Estimate; Population 25 to 34 years - High school graduate or higher
#HC01_EST_VC21------Total; Estimate; Population 25 to 34 years - Bachelor's degree or higher
#HC01_EST_VC23------Total; Estimate; Population 35 to 44 years
#HC01_EST_VC24------Total; Estimate; Population 35 to 44 years - High school graduate or higher
#HC01_EST_VC25------Total; Estimate; Population 35 to 44 years - Bachelor's degree or higher
#HC01_EST_VC27------Total; Estimate; Population 45 to 64 years
#HC01_EST_VC28------Total; Estimate; Population 45 to 64 years - High school graduate or higher
#HC01_EST_VC29------Total; Estimate; Population 45 to 64 years - Bachelor's degree or higher
#HC01_EST_VC31------Total; Estimate; Population 65 years and over
#HC01_EST_VC32------Total; Estimate; Population 65 years and over - High school graduate or higher
#HC01_EST_VC33------Total; Estimate; Population 65 years and over - Bachelor's degree or higher

In [27]:
#Loading educational attainement data
edu = pd.read_csv('educational_attainment.csv')
edu.head()

,GEO.id,GEO.id2,GEO.display-label,HC01_EST_VC01,HC01_MOE_VC01,HC01_EST_VC02,HC01_MOE_VC02,HC01_EST_VC03,HC01_MOE_VC03,HC01_EST_VC04,...,HC01_EST_VC28,HC01_MOE_VC28,HC01_EST_VC29,HC01_MOE_VC29,HC01_EST_VC31,HC01_MOE_VC31,HC01_EST_VC32,HC01_MOE_VC32,HC01_EST_VC33,HC01_MOE_VC33
0,1400000US36005000100,36005000100,"Census Tract 1, Bronx County, New York",2685,361,78.9,3.9,16.1,3.9,5,...,38.7,7.1,5.8,4.3,56,47,14.3,26.4,3.6,9.5
1,1400000US36005000200,36005000200,"Census Tract 2, Bronx County, New York",575,217,35,17.6,19,9.6,29.9,...,79.1,8.2,14.3,7.2,751,177,57.9,11.5,6.7,6.5
2,1400000US36005000400,36005000400,"Census Tract 4, Bronx County, New York",446,121,24,14.8,23.3,10.8,50.4,...,88.4,5.3,33.3,8.2,516,133,55.2,13.5,19.2,10.3
3,1400000US36005001600,36005001600,"Census Tract 16, Bronx County, New York",626,179,19.2,10.3,29.4,17.6,40.6,...,56.1,6.8,8.9,4.6,867,163,52.2,9.2,5.9,5.6
4,1400000US36005001900,36005001900,"Census Tract 19, Bronx County, New York",319,116,32.6,13,28.2,12.8,19.1,...,72,8.6,13.5,8.5,95,45,42.1,27.9,0,26.5


In [28]:
#Adding the following; renaming columns:
#HC01_EST_VC07------Total; Estimate; Population 25 years and over
#HC01_EST_VC16------Total; Estimate; Percent high school graduate or higher
#HC01_EST_VC17------Total; Estimate; Percent bachelor's degree or higher

district_edu = edu[['GEO.id', 'HC01_EST_VC07', 'HC01_EST_VC16', 'HC01_EST_VC17']]
district_edu = district_edu.rename(columns = {'HC01_EST_VC07':'Population 25 years and over', 
                                                 'HC01_EST_VC16':'Percent high school graduate or higher',
                                                 'HC01_EST_VC17':"Percent bachelor's degree or higher"
                                                })

#Adding district

district_edu['district'] = district_edu['GEO.id'].map(districts)
district_edu = district_edu[district_edu['district']>0]
print len(district_edu)
district_edu.head()

2114


,GEO.id,Population 25 years and over,Percent high school graduate or higher,Percent bachelor's degree or higher,district
0,1400000US36005000100,5388,37.3,2.5,7
2,1400000US36005000400,3800,85.3,28.4,8
3,1400000US36005001600,3551,67,14.4,8
4,1400000US36005001900,1642,73.5,18.5,7
5,1400000US36005002000,5488,68.6,14.1,8


In [29]:
#Converting data to float
district_edu['Population 25 years and over'] = map(lambda x: to_number(x), district_edu['Population 25 years and over'])
district_edu['Percent high school graduate or higher'] = map(lambda x: to_number(x), district_edu['Percent high school graduate or higher'])
district_edu["Percent bachelor's degree or higher"] = map(lambda x: to_number(x), district_edu["Percent bachelor's degree or higher"])

print len(district_edu)
district_edu.head()

2114


,GEO.id,Population 25 years and over,Percent high school graduate or higher,Percent bachelor's degree or higher,district
0,1400000US36005000100,5388,37.3,2.5,7
2,1400000US36005000400,3800,85.3,28.4,8
3,1400000US36005001600,3551,67.0,14.4,8
4,1400000US36005001900,1642,73.5,18.5,7
5,1400000US36005002000,5488,68.6,14.1,8


In [30]:
#Calculating totals
district_edu['High school or higher'] = district_edu['Percent high school graduate or higher']/\
100*district_edu['Population 25 years and over']
district_edu["Bachelor's or higher"] = district_edu["Percent bachelor's degree or higher"]/\
100*district_edu['Population 25 years and over']
district_edu = district_edu.groupby('district').sum()

In [31]:
#Calculating percentages
district_edu['Percent high school graduate or higher'] = 100*district_edu['High school or higher']/\
district_edu['Population 25 years and over']
district_edu["Percent bachelor's degree or higher"] = 100*district_edu["Bachelor's or higher"]/\
district_edu['Population 25 years and over']
district_edu

,Population 25 years and over,Percent high school graduate or higher,Percent bachelor's degree or higher,High school or higher,Bachelor's or higher
district,,,,,
1,70373,81.104037,53.416856,57075.344,37591.044
2,484272,93.831138,75.178477,454397.931,364068.316
3,188496,92.018840,69.458436,173451.832,130926.373
4,66273,70.032801,24.784914,46412.838,16425.706
5,84950,78.727037,31.725147,66878.618,26950.512
6,154053,69.419506,29.372073,106942.832,45248.559
7,69463,58.385271,10.946508,40556.161,7603.773
8,128131,71.453672,17.467690,91554.304,22381.526
9,139501,62.548160,11.585467,87255.308,16161.842


In [45]:
#Calculating aggregate ratios

socioeconomic['Percent high school graduate or higher'] = np.zeros(32)
for i in range(1,33):
    socioeconomic['Percent high school graduate or higher'][i] = \
    district_edu['Percent high school graduate or higher'][i]

socioeconomic["Percent bachelor's degree or higher"] = np.zeros(32)
for i in range(1,33):
    socioeconomic["Percent bachelor's degree or higher"][i] = \
    district_edu["Percent bachelor's degree or higher"][i] 

socioeconomic

,Median income (dollars),Percent high school graduate or higher,Percent bachelor's degree or higher
District,,,
1,62768.0,81.104037,53.416856
2,106625.0,93.831138,75.178477
3,88067.5,92.018840,69.458436
4,26578.0,70.032801,24.784914
5,35583.0,78.727037,31.725147
6,36999.5,69.419506,29.372073
7,21959.0,58.385271,10.946508
8,44840.0,71.453672,17.467690
9,24134.5,62.548160,11.585467


#Employment characteristics

In [33]:
#HC01_EST_VC01------Total; Estimate; Families
#HC02_EST_VC01------Families with own children under 18 years; Estimate; Families
#HC01_EST_VC03------Total; Estimate; EMPLOYMENT STATUS CHARACTERISTICS - Married-couple families
#HC02_EST_VC03------Families with own children under 18 years; Estimate; EMPLOYMENT STATUS CHARACTERISTICS - Married-couple families
#HC01_EST_VC04------Total; Estimate; EMPLOYMENT STATUS CHARACTERISTICS - Married-couple families - Both husband and wife in labor force
#HC02_EST_VC04------Families with own children under 18 years; Estimate; EMPLOYMENT STATUS CHARACTERISTICS - Married-couple families - Both husband and wife in labor force
#HC01_EST_VC05------Total; Estimate; EMPLOYMENT STATUS CHARACTERISTICS - Married-couple families - Husband in labor force, wife not in labor force
#HC02_EST_VC05------Families with own children under 18 years; Estimate; EMPLOYMENT STATUS CHARACTERISTICS - Married-couple families - Husband in labor force, wife not in labor force
#HC01_EST_VC06------Total; Estimate; EMPLOYMENT STATUS CHARACTERISTICS - Married-couple families - Wife in labor force, husband not in labor force
#HC02_EST_VC06------Families with own children under 18 years; Estimate; EMPLOYMENT STATUS CHARACTERISTICS - Married-couple families - Wife in labor force, husband not in labor force
#HC01_EST_VC07------Total; Estimate; EMPLOYMENT STATUS CHARACTERISTICS - Married-couple families - Both husband and wife not in labor force
#HC02_EST_VC07------Families with own children under 18 years; Estimate; EMPLOYMENT STATUS CHARACTERISTICS - Married-couple families - Both husband and wife not in labor force
#HC01_EST_VC09------Total; Estimate; Other families
#HC02_EST_VC09------Families with own children under 18 years; Estimate; Other families
#HC01_EST_VC10------Total; Estimate; Other families - Female householder, no husband present
#HC02_EST_VC10------Families with own children under 18 years; Estimate; Other families - Female householder, no husband present
#HC01_EST_VC11------Total; Estimate; Other families - Female householder, no husband present - In labor force
#HC02_EST_VC11------Families with own children under 18 years; Estimate; Other families - Female householder, no husband present - In labor force
#HC01_EST_VC12------Total; Estimate; Other families - Female householder, no husband present - Not in labor force
#HC02_EST_VC12------Families with own children under 18 years; Estimate; Other families - Female householder, no husband present - Not in labor force
#HC01_EST_VC13------Total; Estimate; Other families - Male householder, no wife present
#HC02_EST_VC13------Families with own children under 18 years; Estimate; Other families - Male householder, no wife present
#HC01_EST_VC14------Total; Estimate; Other families - Male householder, no wife present - In labor force
#HC02_EST_VC14------Families with own children under 18 years; Estimate; Other families - Male householder, no wife present - In labor force
#HC01_EST_VC15------Total; Estimate; Other families - Male householder, no wife present - Not in labor force
#HC02_EST_VC15------Families with own children under 18 years; Estimate; Other families - Male householder, no wife present - Not in labor force
#HC01_EST_VC18------Total; Estimate; WORK STATUS CHARACTERISTICS - Families
#HC02_EST_VC18------Families with own children under 18 years; Estimate; WORK STATUS CHARACTERISTICS - Families
#HC01_EST_VC19------Total; Estimate; WORK STATUS CHARACTERISTICS - Families - No workers in the past 12 months
#HC02_EST_VC19------Families with own children under 18 years; Estimate; WORK STATUS CHARACTERISTICS - Families - No workers in the past 12 months
#HC01_EST_VC20------Total; Estimate; WORK STATUS CHARACTERISTICS - Families - 1 worker in the past 12 months
#HC02_EST_VC20------Families with own children under 18 years; Estimate; WORK STATUS CHARACTERISTICS - Families - 1 worker in the past 12 months
#HC01_EST_VC21------Total; Estimate; WORK STATUS CHARACTERISTICS - Families - 2 or more workers in the past 12 months
#HC02_EST_VC21------Families with own children under 18 years; Estimate; WORK STATUS CHARACTERISTICS - Families - 2 or more workers in the past 12 months
#HC01_EST_VC23------Total; Estimate; Married-couple families
#HC02_EST_VC23------Families with own children under 18 years; Estimate; Married-couple families
#HC01_EST_VC24------Total; Estimate; Married-couple families - Householder worked full-time, year-round in the past 12 months
#HC02_EST_VC24------Families with own children under 18 years; Estimate; Married-couple families - Householder worked full-time, year-round in the past 12 months
#HC01_EST_VC25------Total; Estimate; Married-couple families - Householder worked full-time, year-round in the past 12 months - Spouse worked full-time, year-round in the past 12 months
#HC02_EST_VC25------Families with own children under 18 years; Estimate; Married-couple families - Householder worked full-time, year-round in the past 12 months - Spouse worked full-time, year-round in the past 12 months
#HC01_EST_VC26------Total; Estimate; Married-couple families - Householder worked part-time or part-year in the past 12 months
#HC02_EST_VC26------Families with own children under 18 years; Estimate; Married-couple families - Householder worked part-time or part-year in the past 12 months
#HC01_EST_VC27------Total; Estimate; Married-couple families - Householder worked part-time or part-year in the past 12 months - Spouse worked part-time or part-year in the past 12 months
#HC02_EST_VC27------Families with own children under 18 years; Estimate; Married-couple families - Householder worked part-time or part-year in the past 12 months - Spouse worked part-time or part-year in the past 12 months
#HC01_EST_VC28------Total; Estimate; Married-couple families - Householder did not work in the past 12 months
#HC02_EST_VC28------Families with own children under 18 years; Estimate; Married-couple families - Householder did not work in the past 12 months
#HC01_EST_VC29------Total; Estimate; Married-couple families - Householder did not work in the past 12 months - Spouse did not work in the past 12 months
#HC02_EST_VC29------Families with own children under 18 years; Estimate; Married-couple families - Householder did not work in the past 12 months - Spouse did not work in the past 12 months

In [34]:
#Loading employment characteristics file
employment_characteristics = pd.read_csv('employment_characteristics.csv')
employment_characteristics.head()

,GEO.id,GEO.id2,GEO.display-label,HC01_EST_VC01,HC01_MOE_VC01,HC02_EST_VC01,HC02_MOE_VC01,HC01_EST_VC03,HC01_MOE_VC03,HC02_EST_VC03,...,HC02_EST_VC27,HC02_MOE_VC27,HC01_EST_VC28,HC01_MOE_VC28,HC02_EST_VC28,HC02_MOE_VC28,HC01_EST_VC29,HC01_MOE_VC29,HC02_EST_VC29,HC02_MOE_VC29
0,1400000US36005000100,36005000100,"Census Tract 1, Bronx County, New York",0,16,0,16,0,16,0,...,-,**,-,**,-,**,-,**,-,**
1,1400000US36005000200,36005000200,"Census Tract 2, Bronx County, New York",1166,94,507,121,650,123,268,...,8.2,10.7,36,13.8,25.7,21.3,16.3,9.4,0,10.6
2,1400000US36005000400,36005000400,"Census Tract 4, Bronx County, New York",1442,134,672,147,633,136,263,...,4.6,6.6,22.5,10.8,0,10.8,16.5,9.5,0,10.8
3,1400000US36005001600,36005001600,"Census Tract 16, Bronx County, New York",1316,135,551,119,610,117,208,...,4.8,8.6,36.4,11.6,9.1,13.9,22.6,9.6,0,13.4
4,1400000US36005001900,36005001900,"Census Tract 19, Bronx County, New York",468,83,290,70,201,64,106,...,0,24.3,25.9,14.3,23.6,20.1,3,4.7,0,24.3


In [35]:
#Adding the following; renaming columns:
#HC02_EST_VC01------Families with own children under 18 years; Estimate; Families
#HC02_EST_VC19------Families with own children under 18 years; Estimate; No workers in the past 12 months
#HC02_EST_VC20------Families with own children under 18 years; Estimate; 1 worker in the past 12 months
#HC02_EST_VC21------Families with own children under 18 years; Estimate; 2 or more workers in the past 12 months
district_employment = employment_characteristics[['GEO.id', 'HC02_EST_VC01', 'HC02_EST_VC19', 
                                                  'HC02_EST_VC20', 'HC02_EST_VC21']]
district_employment = district_employment.rename(columns = {'HC02_EST_VC01':'Families with own children under 18 years', 
                                                     'HC02_EST_VC19':'No workers in the past 12 months',
                                                     'HC02_EST_VC20':'1 worker in the past 12 months', 
                                                     'HC02_EST_VC21':'2 or more workers in the past 12 months'
                                            
                                                })

#Adding district

district_employment['district'] = district_employment['GEO.id'].map(districts)
district_employment = district_employment[district_employment['district']>0]
print len(district_employment)
district_employment.head()

2114


,GEO.id,Families with own children under 18 years,No workers in the past 12 months,1 worker in the past 12 months,2 or more workers in the past 12 months,district
0,1400000US36005000100,0,-,-,-,7
2,1400000US36005000400,672,20.7,26.8,52.5,8
3,1400000US36005001600,551,7.6,48.3,44.1,8
4,1400000US36005001900,290,16.9,54.5,28.6,7
5,1400000US36005002000,1209,33.7,55.5,10.8,8


In [36]:
#Converting data to float
district_employment['Families with own children under 18 years'] = \
map(lambda x: to_number(x), district_employment['Families with own children under 18 years'])
district_employment['No workers in the past 12 months'] = \
map(lambda x: to_number(x), district_employment['No workers in the past 12 months'])
district_employment['1 worker in the past 12 months'] = \
map(lambda x: to_number(x), district_employment['1 worker in the past 12 months'])
district_employment['2 or more workers in the past 12 months'] = \
map(lambda x: to_number(x), district_employment['2 or more workers in the past 12 months'])

district_employment.head()

,GEO.id,Families with own children under 18 years,No workers in the past 12 months,1 worker in the past 12 months,2 or more workers in the past 12 months,district
0,1400000US36005000100,0,0.0,0.0,0.0,7
2,1400000US36005000400,672,20.7,26.8,52.5,8
3,1400000US36005001600,551,7.6,48.3,44.1,8
4,1400000US36005001900,290,16.9,54.5,28.6,7
5,1400000US36005002000,1209,33.7,55.5,10.8,8


In [37]:
#Calculating totals
district_employment['No workers in the past 12 months (total)'] = district_employment['No workers in the past 12 months']/\
100*district_employment['Families with own children under 18 years']
district_employment['1 worker in the past 12 months (total)'] = district_employment['1 worker in the past 12 months']/\
100*district_employment['Families with own children under 18 years']
district_employment['2 or more workers in the past 12 months (total)'] = district_employment['2 or more workers in the past 12 months']/\
100*district_employment['Families with own children under 18 years']
district_employment = district_employment.groupby('district').sum()

In [38]:
district_employment

,Families with own children under 18 years,No workers in the past 12 months,1 worker in the past 12 months,2 or more workers in the past 12 months,No workers in the past 12 months (total),1 worker in the past 12 months (total),2 or more workers in the past 12 months (total)
district,,,,,,,
1,5092,117.7,907.0,675.2,420.700,2639.754,2031.660
2,39676,489.6,4889.4,5921.3,1374.997,17331.734,20972.498
3,21666,226.0,1565.4,1808.7,1160.576,9452.651,11053.123
4,9361,311.9,898.7,489.4,1760.054,4974.195,2627.371
5,11858,363.2,1362.6,774.6,2006.184,6297.067,3557.193
6,17211,253.5,1299.8,1446.6,1389.693,7866.388,7953.909
7,14165,508.5,1306.1,685.5,3031.034,7399.788,3735.417
8,22986,658.9,2259.5,1781.9,3616.602,11462.052,7909.603
9,30650,818.2,2828.8,1553.0,4762.952,16810.645,9075.420


In [39]:
district_employment['No workers in the past 12 months'] = 100*district_employment['No workers in the past 12 months (total)']/\
district_employment['Families with own children under 18 years']
district_employment['1 worker in the past 12 months'] = 100*district_employment['1 worker in the past 12 months (total)']/\
district_employment['Families with own children under 18 years']
district_employment['2 or more workers in the past 12 months'] = 100*district_employment['2 or more workers in the past 12 months (total)']/\
district_employment['Families with own children under 18 years']
district_employment

,Families with own children under 18 years,No workers in the past 12 months,1 worker in the past 12 months,2 or more workers in the past 12 months,No workers in the past 12 months (total),1 worker in the past 12 months (total),2 or more workers in the past 12 months (total)
district,,,,,,,
1,5092,8.261980,51.841202,39.899057,420.700,2639.754,2031.660
2,39676,3.465564,43.683169,52.859406,1374.997,17331.734,20972.498
3,21666,5.356669,43.628962,51.015984,1160.576,9452.651,11053.123
4,9361,18.801987,53.137432,28.067204,1760.054,4974.195,2627.371
5,11858,16.918401,53.103955,29.998254,2006.184,6297.067,3557.193
6,17211,8.074447,45.705584,46.214101,1389.693,7866.388,7953.909
7,14165,21.398052,52.239944,26.370752,3031.034,7399.788,3735.417
8,22986,15.733934,49.865362,34.410524,3616.602,11462.052,7909.603
9,30650,15.539811,54.847129,29.609853,4762.952,16810.645,9075.420


In [47]:
socioeconomic['No workers in the past 12 months (%)'] = np.zeros(32)
for i in range(1,33):
    socioeconomic['No workers in the past 12 months (%)'][i] = \
    district_employment['No workers in the past 12 months'][i]

socioeconomic['1 worker in the past 12 months (%)'] = np.zeros(32)
for i in range(1,33):
    socioeconomic['1 worker in the past 12 months (%)'][i] = \
    district_employment['1 worker in the past 12 months'][i]
    
socioeconomic['2 or more workers in the past 12 months (%)'] = np.zeros(32)
for i in range(1,33):
    socioeconomic['2 or more workers in the past 12 months (%)'][i] = \
    district_employment['2 or more workers in the past 12 months'][i]

socioeconomic

,Median income (dollars),Percent high school graduate or higher,Percent bachelor's degree or higher,No workers in the past 12 months (%),1 worker in the past 12 months (%),2 or more workers in the past 12 months (%)
District,,,,,,
1,62768.0,81.104037,53.416856,8.261980,51.841202,39.899057
2,106625.0,93.831138,75.178477,3.465564,43.683169,52.859406
3,88067.5,92.018840,69.458436,5.356669,43.628962,51.015984
4,26578.0,70.032801,24.784914,18.801987,53.137432,28.067204
5,35583.0,78.727037,31.725147,16.918401,53.103955,29.998254
6,36999.5,69.419506,29.372073,8.074447,45.705584,46.214101
7,21959.0,58.385271,10.946508,21.398052,52.239944,26.370752
8,44840.0,71.453672,17.467690,15.733934,49.865362,34.410524
9,24134.5,62.548160,11.585467,15.539811,54.847129,29.609853


In [48]:
#Pickling
pickle.dump(socioeconomic, open("socioeconomic.p", "wb" ))